In [1]:
import os
import re
import json
import gzip
import wget
import pandas as pd
import numpy as np
from urllib.request import urlopen
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import time
import nltk
from imp import reload

#cleaning textfiles libraries
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
#import pyLDAvis.gensim # To visualise LDA model effectively

C:\Users\Triston\AppData\Local\Temp\ipykernel_9580\2848255064.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
##download data from url
### randomly selected file to model
url = 'https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz'
filename = wget.download(url)

In [3]:
#load metadata
data = []
with gzip.open('Grocery_and_Gourmet_Food_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

1143860
{'overall': 5.0, 'verified': True, 'reviewTime': '11 19, 2014', 'reviewerID': 'A1QVBUH9E1V6I8', 'asin': '4639725183', 'reviewerName': 'Jamshed Mathur', 'reviewText': 'No adverse comment.', 'summary': 'Five Stars', 'unixReviewTime': 1416355200}


In [4]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

df_initial = df

1143860


In [5]:
#look at dataframe
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143860 entries, 0 to 1143859
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1143860 non-null  float64
 1   verified        1143860 non-null  bool   
 2   reviewTime      1143860 non-null  object 
 3   reviewerID      1143860 non-null  object 
 4   asin            1143860 non-null  object 
 5   reviewerName    1143722 non-null  object 
 6   reviewText      1143470 non-null  object 
 7   summary         1143641 non-null  object 
 8   unixReviewTime  1143860 non-null  int64  
 9   vote            158202 non-null   object 
 10  style           592086 non-null   object 
 11  image           9510 non-null     object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 97.1+ MB


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1143855,5.0,True,"09 8, 2017",A223YRQH2Z5T1D,B01HJF6FRA,flint5292,"As a new vegan, it is sometimes difficult to r...","As a new vegan, it is sometimes difficult to ...",1504828800,4,NaN,NaN
1143856,5.0,True,"08 4, 2017",A38GDA4TB9EILT,B01HJF6FRA,Moriah Bolyard,The best thing ever is ordering a product you ...,The best thing ever is ordering a product you ...,1501804800,3,NaN,NaN
1143857,5.0,True,"07 4, 2017",A2025PN7HDC5BO,B01HJF6FRA,M.C,I used to love ranch before I became vegan. It...,Just what the vegan ordered!,1499126400,5,NaN,NaN
1143858,5.0,True,"06 7, 2017",A1NY7XWC7EPQOA,B01HJF6FRA,Greensboro,I cannot have dairy nor gluten. This is as cl...,This is as close to Ranch as I will ever be ab...,1496793600,2,NaN,NaN


In [6]:
#drop columns that don't impact
df = df.drop(['style','summary','image'], axis=1)

In [7]:
#convert vote column to float

df['vote']=df['vote'].str.replace(',','')
df["vote"]= df["vote"].fillna(0)
df["vote"] = df["vote"].astype(float)

In [8]:
#initial df - setting the vote column to float to create the helpful column

df_initial['vote']=df_initial['vote'].str.replace(',','')
df_initial["vote"]= df_initial["vote"].fillna(0)
df_initial["vote"] = df_initial["vote"].astype(float)

In [9]:
#convert column to string
df["reviewText"]=df["reviewText"].astype(str)
df_initial["reviewText"]=df_initial["reviewText"].astype(str)

In [10]:
## inserting helpful flag to be used in EDA and Models
df['helpful_flag'] = np.where(df['vote'] > 0, 1, 0)
df_initial['helpful_flag'] = np.where(df_initial['vote'] > 0, 1, 0)

In [11]:
#Determine Average Review Length and add review length column to dataframe

x = [len(df['reviewText'][i]) for i in range(df['reviewText'].shape[0])]
print('average length of review: {:.3f}'.format(sum(x)/len(x)) )

df['totalWords'] = df['reviewText'].str.split().str.len()

average length of review: 208.585


In [12]:
#check dataframe after add and changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143860 entries, 0 to 1143859
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1143860 non-null  float64
 1   verified        1143860 non-null  bool   
 2   reviewTime      1143860 non-null  object 
 3   reviewerID      1143860 non-null  object 
 4   asin            1143860 non-null  object 
 5   reviewerName    1143722 non-null  object 
 6   reviewText      1143860 non-null  object 
 7   unixReviewTime  1143860 non-null  int64  
 8   vote            1143860 non-null  float64
 9   helpful_flag    1143860 non-null  int32  
 10  totalWords      1143860 non-null  int64  
dtypes: bool(1), float64(2), int32(1), int64(2), object(5)
memory usage: 84.0+ MB


### Removing Duplicate Reviews

In [13]:
duplicated_reviews = df.duplicated(subset=["reviewerID","reviewTime","reviewText"], keep='first') #returns a Series with True and False values that describe which rows in the DataFrame are duplicated and not.
count_duplicated_reviews = duplicated_reviews.value_counts()

sum_reviews = count_duplicated_reviews.sum()
perc_duplicated_reviews = (count_duplicated_reviews/sum_reviews) * 100

x = PrettyTable()
x.field_names = ["","Count","Percentage of Total"]
x.add_rows([
    ["Duplicate Reviews", count_duplicated_reviews[True], perc_duplicated_reviews[True]],
    ["Original Reviews", count_duplicated_reviews[False], perc_duplicated_reviews[False]],
])
print(x)

+-------------------+--------+---------------------+
|                   | Count  | Percentage of Total |
+-------------------+--------+---------------------+
| Duplicate Reviews | 174460 |  15.25186648715752  |
|  Original Reviews | 969400 |  84.74813351284249  |
+-------------------+--------+---------------------+


In [14]:
df_duplicates = df[duplicated_reviews]
df_duplicates.sort_values(by = ['totalWords'], ascending = [False])
df_duplicates[df_duplicates['vote'] == 2.0]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,unixReviewTime,vote,helpful_flag,totalWords
2744,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B0000DI0KA,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,2.0,1,5
4479,5.0,True,"03 6, 2014",A2EYQXB9N8ZWRW,B0000SXENM,Virginia Davenport,I am one of those people who love sweets and a...,1394064000,2.0,1,105
7457,5.0,False,"10 27, 2014",AL0V2MSXOFO61,B00016LA8Y,ECS,It was nice to get this while deployed to a FO...,1414368000,2.0,1,13
8672,5.0,False,"05 28, 2012",A21BJAFOZL0M35,B000168QTU,Kennedy,Flavorful bursts of cinnamon and dashes of lic...,1338163200,2.0,1,334
8673,5.0,False,"05 28, 2012",A21BJAFOZL0M35,B000168QTU,Kennedy,Flavorful bursts of cinnamon and dashes of lic...,1338163200,2.0,1,334
...,...,...,...,...,...,...,...,...,...,...,...
1141204,5.0,True,"05 7, 2018",AL36E6FCP20DA,B01DJQA8RQ,Janays,"fast delivery, great price and quality",1525651200,2.0,1,6
1142270,5.0,True,"03 10, 2018",A32PI21UL0YVU9,B01F7MCTBS,Kevin R. Seiter,Love their products. I order lots of their dri...,1520640000,2.0,1,23
1142272,5.0,True,"05 13, 2017",A1CWGDRJDELFSD,B01F7MCTBS,Amazon Customer,Consistently great product! Arrives ahead of ...,1494633600,2.0,1,10
1143691,3.0,True,"08 30, 2017",AXCNLJZ6ME8PM,B01H73VY7K,Claudia K,Disappointing flavor. Celery root has such a d...,1504051200,2.0,1,35


In [15]:
df_new = df[(df['reviewerID'] == 'A2N8B21NWXHIW7') & (df['unixReviewTime'] == 1469145600) ]
df_new

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,unixReviewTime,vote,helpful_flag,totalWords
779,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B0000A9XZO,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
2744,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B0000DI0KA,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,2.0,1,5
163722,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B000SWTKV0,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
289820,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B001B49UOG,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
330617,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B001GVIRYS,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
341247,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B001LO511W,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
372815,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B00269YPBS,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
397607,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B002MBKF0U,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
432665,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B003VTI0YK,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5
557898,5.0,True,"07 22, 2016",A2N8B21NWXHIW7,B006AXEO0S,Robbi L Shults,"excellent product, wonderful transaction. thanks!",1469145600,0.0,0,5


In [16]:
#remove duplicate reviews for df
df = df[~duplicated_reviews]

print(f"Number of reviews after removel of duplicates : {df.shape[0]}")

Number of reviews after removel of duplicates : 969400


### Helpfulness vs Word Count

In [17]:
#Group by word count and sort

df2= df[["totalWords","helpful_flag"]].sort_values(['totalWords']).groupby(["totalWords", "helpful_flag"])['helpful_flag'].count().reset_index(name='count')

df2

,totalWords,helpful_flag,count
0,1,0,32032
1,1,1,343
2,2,0,48772
3,2,1,568
4,3,0,31564
...,...,...,...
1743,2088,0,1
1744,2095,1,1
1745,2171,0,1
1746,2283,1,1


In [18]:
#Pivot
df3 = (df2.pivot('totalWords', 'helpful_flag')
        .stack(level=0)
        .rename_axis(index=['totalWords', 'count'], columns=None)
    ).reset_index()

df3 = df3.fillna(0)

df3 = df3[["totalWords",0,1]]

df3

C:\Users\Triston\AppData\Local\Temp\ipykernel_9580\84754139.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df3 = (df2.pivot('totalWords', 'helpful_flag')


,totalWords,0,1
0,1,32032.0,343.0
1,2,48772.0,568.0
2,3,31564.0,527.0
3,4,32244.0,679.0
4,5,28600.0,655.0
...,...,...,...
1036,2088,1.0,0.0
1037,2095,0.0,1.0
1038,2171,1.0,0.0
1039,2283,0.0,1.0


In [19]:
df3["totalFlag"] = df3[[0,1]].sum(axis=1)
df3

,totalWords,0,1,totalFlag
0,1,32032.0,343.0,32375.0
1,2,48772.0,568.0,49340.0
2,3,31564.0,527.0,32091.0
3,4,32244.0,679.0,32923.0
4,5,28600.0,655.0,29255.0
...,...,...,...,...
1036,2088,1.0,0.0,1.0
1037,2095,0.0,1.0,1.0
1038,2171,1.0,0.0,1.0
1039,2283,0.0,1.0,1.0


In [29]:
#Add cummulative %
df3["Cumulative % - 0"] = ((df3[0].cumsum())/(df3[0].sum()))*100
df3["Cumulative % - 1"] = ((df3[1].cumsum())/(df3[1].sum()))*100

#Labeling X1 and X2 - Need to change label
df3["X1"] = df3[0] * df3["totalWords"]
df3["X2"] = df3[1] * df3["totalWords"]

meanX1 = (df3["X1"].sum())/(df3[0].sum())
meanX2 = (df3["X2"].sum())/(df3[1].sum())

df3

#print(meanX1)
#print(meanX2)

,totalWords,0,1,totalFlag,Cumulative % - 0,Cumulative % - 1,X1,X2
0,1,32032.0,343.0,32375.0,3.878774,0.238905,32032.0,343.0
1,2,48772.0,568.0,49340.0,9.784604,0.634525,97544.0,1136.0
2,3,31564.0,527.0,32091.0,13.606707,1.001588,94692.0,1581.0
3,4,32244.0,679.0,32923.0,17.511152,1.474521,128976.0,2716.0
4,5,28600.0,655.0,29255.0,20.974343,1.930739,143000.0,3275.0
...,...,...,...,...,...,...,...,...
1036,2088,1.0,0.0,1.0,99.999879,99.997910,2088.0,0.0
1037,2095,0.0,1.0,1.0,99.999879,99.998607,0.0,2095.0
1038,2171,1.0,0.0,1.0,100.000000,99.998607,2171.0,0.0
1039,2283,0.0,1.0,1.0,100.000000,99.999303,0.0,2283.0


In [27]:
#Histogram of total words

fig = px.histogram(df3, x="totalWords")

fig.show()

In [26]:
#Cumulative % Chart

df_chart = df3.melt(id_vars='totalWords', value_vars=['Cumulative % - 0', 'Cumulative % - 1'])

df_chart

fig = px.line(df_chart, x='totalWords' , y='value' , color='variable')

fig.update_layout(
    title="Word Count vs Cumulative Percentage of Helpfulness Count",
    legend_title="Cummulative Category",
    xaxis_title="Total Words",
    yaxis_title="Percent"
)

fig.update_xaxes(range=[0, 400])

fig.show()

### T-Test on helpfulness

In [51]:
import scipy.stats as st

# Perform the two sample t-test with equal variances
T = st.ttest_ind(a=df3["X1"], b=df3["X2"], equal_var=True)
T

Ttest_indResult(statistic=6.892510833785069, pvalue=7.241203044729252e-12)

In [35]:
# Calculate confidence intervals for X1
ci = st.t.interval(0.95, len(df3["X1"])-1, loc=np.mean(df3["X1"]), scale=st.sem(df3["X1"]))
print("Confidence interval X1: ", ci)

Confidence interval X1:  (21659.0798169533, 28671.492709463608)


In [34]:
# Calculate confidence intervals for X1
ci = st.t.interval(0.95, len(df3["X2"])-1, loc=np.mean(df3["X2"]), scale=st.sem(df3["X2"]))
print("Confidence interval X2: ", ci)

Confidence interval X2:  (11217.678781633684, 13358.791919615116)
